#### This notebook showcases the workflow of segmenting (one of) the NFH staining that accompanies the Resolve data.

Steps that precede the running of this notebook:

1.    Segment the NFH staining using QuPath, and running the src/segment_nfh.groovy macro
2.    While still in QuPath, export the resulting mask to ImageJ, and run the src/make_labeled_image.ijm macro to turn the segmnentation mask into a labeled image, where each integer represents a separate object.

3.   (Optional) add column names to the results.txt file of the Resolve technique for easy parsing with pandas (col, row, z, gene)

In [5]:
# use environment created based on yamls/img_processing.yml
from skimage import io
from skimage.measure import label
from src.make_count_matrices_from_labeled_image import createGeneMatrix

# For segmenting the tissue
from skimage.filters import threshold_li
from skimage.morphology import convex_hull_image

In [ ]:
# NFH segmented image was created by applying the src/segment_nfh.groovy macro to the NFH staining
nfh_labeled_image = io.imread("/path/to/labeled_image.tif")

resolve_results = pd.read_csv("/path/to/resolve_results_for_this_slide.txt")

In [ ]:
# To make the mask of what ISN'T nfh, we could just invert the mask. However, some off-tissue genes are sometimes found
# To also filter these out, we take the original NFH staining and segment the entire tissue, to also invert that mask
original_nfh_staining = io.imread("/path/to/nfh_staining.tif")

thresh = threshold_li(original_nfh_staining)
tissue_mask = original_nfh_staining > thresh
tissue_mask = convex_hull_image(tissue_mask)

negative_labeled_image = label((nfh_labeled_image == 0) & (tissue_mask))

In [ ]:
nfh_count_matrix = createGeneMatrix(nfh_labeled_image, resolve_results)
negative_count_matrix = createGeneMatrix(negative_labeled_image, resolve_results)